# Author: Riya Nakarmi ###<br>
# College Project ###

In [24]:
import random
import json
import pickle
import numpy as np

In [25]:

from nltk.stem import WordNetLemmatizer

In [26]:
import nltk


In [27]:
nltk.download('popular')


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
# read the json file in using pandas

In [29]:
import pandas as pd

In [30]:
lemmatizer = WordNetLemmatizer()

In [31]:
intents = pd.read_json("/intents.json")

In [ ]:
intents.head()

,intents
0,"{'tag': 'greetings', 'patterns': ['hello', 'he..."
1,"{'tag': 'name', 'patterns': ['what is your nam..."
2,"{'tag': 'courses', 'patterns': ['what courses ..."
3,"{'tag': 'courseDuration', 'patterns': ['how lo..."
4,"{'tag': 'Location', 'patterns': ['location', '..."


In [32]:
intents

,intents
0,"{'tag': 'greetings', 'patterns': ['hello', 'he..."
1,"{'tag': 'name', 'patterns': ['what is your nam..."
2,"{'tag': 'courses', 'patterns': ['what courses ..."
3,"{'tag': 'courseDuration', 'patterns': ['how lo..."
4,"{'tag': 'Location', 'patterns': ['location', '..."
5,"{'tag': 'semesters', 'patterns': ['how many se..."
6,"{'tag': 'semDuration', 'patterns': ['how many ..."
7,"{'tag': 'studentRequirements', 'patterns': ['w..."
8,"{'tag': 'classes', 'patterns': ['how many clas..."
9,"{'tag': 'teachingStyle', 'patterns': ['what is..."


In [33]:
intents = json.loads(open('/intents.json').read())

In [34]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!',',','.']

In [35]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [36]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [37]:
classes = sorted(set(classes))

In [38]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [39]:
training = []
output_empty = [0] * len(classes)

In [40]:
for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [41]:
random.shuffle(training)

# Separate features (train_x) and labels (train_y)
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

In [42]:
train_x = list(train_x)
train_y = list(train_y)


In [43]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [44]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)


Epoch 1/200
12/12 [==============================] - 1s 3ms/step - loss: 2.9690 - accuracy: 0.0000e+00
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 2.8136 - accuracy: 0.1724
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 2.7684 - accuracy: 0.2241
Epoch 4/200
12/12 [==============================] - 0s 3ms/step - loss: 2.7419 - accuracy: 0.1552
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 2.5893 - accuracy: 0.1897
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 2.6081 - accuracy: 0.1034
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 2.5008 - accuracy: 0.2586
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 2.2706 - accuracy: 0.3966
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 2.3236 - accuracy: 0.2931
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 2.1038 - accuracy: 0.3448
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
print('Done')

Done
